In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
from nltk.corpus import stopwords
from nltk import word_tokenize
import matplotlib.pyplot as plt

In [ ]:
import wordcloud
import string

In [ ]:
sms = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding='ISO-8859-1')

In [ ]:
sms.head(10)

In [ ]:
sms.columns=['label','text','A','B','C']

In [ ]:
sms.drop(['A','B','C'],axis=1,inplace=True)

In [ ]:
sms.head(6)

In [ ]:
sms.groupby('label').describe()

In [ ]:
pd.value_counts(sms['label'])

In [ ]:
sns.countplot(x=sms['label'],data=sms)

In [ ]:
sms['label']=sms['label'].map({'ham':0,'spam':1}).astype(int)

In [ ]:
sms.head(10)

PREPROCESSING OF DATA

In [ ]:
sms.describe()

In [ ]:
stop_words=set(stopwords.words("english"))

In [ ]:
sms['text'].iloc[5571]

In [ ]:
#words

In [ ]:
def text_preprocess(text):
    words=text.lower().split()
    actual_list=[]
    for word in words:
        if word not in stop_words:
            if word not in string.punctuation:
                actual_list.append(word)
    return(' '.join(actual_list))            

In [ ]:
sms['text'].head().apply(text_preprocess)

Quick Look at the words using Word Cloud


In [ ]:
data_ham  = sms[sms['label'] == 0].copy()
data_spam = sms[sms['label'] == 1].copy()

In [ ]:
def show_wordcloud(data_spam_or_ham, header):
    text = ' '.join(data_spam_or_ham['text'].astype(str).tolist())
    stopwords = set(wordcloud.STOPWORDS)
    
    fig_wordcloud = wordcloud.WordCloud(stopwords=stopwords,background_color='lightgrey',
                    colormap='viridis', width=800, height=600).generate(text)
    
    plt.figure(figsize=(12,6), frameon=True)
    plt.imshow(fig_wordcloud)  
    plt.axis('off')
    plt.title(header, fontsize=20 )
    plt.show()

In [ ]:
show_wordcloud(data_spam, "SPAM Words")

In [ ]:
show_wordcloud(data_ham, "HAM Words")

FEATURE EXTRACTION

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
Cnt_Vector = CountVectorizer(analyzer=text_preprocess)
SMS_TEXTS=Cnt_Vector.fit_transform(sms['text'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(SMS_TEXTS, sms['label'], test_size=0.20, random_state=0)

In [ ]:
print(X_train)

In [ ]:
# Create and train the naive Bayes classifier
# The multinomial Naive Bayes classifier is suitable for classification with discrete features .We shall explore others as we go ahead

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train, y_train)

In [ ]:
# Evaluate the model on the training data set

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train, pred))
print()
print('Confusion Matrix:\n',confusion_matrix(y_train, pred))
print()
print('Model Accuracy : ',accuracy_score(y_train, pred))

In [ ]:
#Evaluate the model on the test data set

pred = classifier.predict(X_test)
print(classification_report(y_test, pred))
print()
print('Confusion Matrix:\n',confusion_matrix(y_test, pred))
print()
print('Model Accuracy : ',accuracy_score(y_test, pred))

Things to do: Verifying other algrithms to see if they work better.Using a TFIDF vectorizer instead of countvectorizer

In [ ]:
test=['Your number as 10000']
x = Cnt_Vector.transform(test)
classifier.predict(x)[0]